In [1]:
#from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
import networkx as nx
from torch import optim
import scipy.sparse as sp
import torch.nn as nn
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import time
import torch

In [2]:
#nnz ne fonctionne pas correctement
#need to take diago superior of nnz because adj symmetric

def load_data(): 
    """
    Function that loads graphs
    """  
    graph_indicator = np.loadtxt("../data/PROT_graph_indicator.txt", dtype=np.int64)
    _, graph_size = np.unique(graph_indicator, return_counts=True)
    
    edges = np.loadtxt("../data/edgelist.txt", dtype=np.int64, delimiter=",")
    A = sp.csr_matrix((np.ones(edges.shape[0]), (edges[:,0], edges[:,1])), shape=(graph_indicator.size, graph_indicator.size))
    A += A.T
    x = np.loadtxt("../data/PROT_node_attributes.txt", delimiter=",")
    edge_attr = np.loadtxt("../data/PROT_edge_attributes.txt", delimiter=",")
    
    adj = []
    features = []
    edge_features = []
    edge_per_graph = []
    idx_n = 0
    idx_m = 0
    for i in range(graph_size.size):
        adj.append(A[idx_n:idx_n+graph_size[i],idx_n:idx_n+graph_size[i]])
        edge_a_triu = len(np.where(adj[i][np.triu_indices(adj[i].shape[0])])[0]!=0)
        edge_features.append(edge_attr[idx_m:idx_m+edge_a_triu,:])
        features.append(x[idx_n:idx_n+graph_size[i],:])
        edge_per_graph.append(edge_a_triu)
        idx_n += graph_size[i]
        idx_m += edge_a_triu

    return edge_per_graph, adj, features, edge_features

def normalize_adjacency(A):
    """
    Function that normalizes an adjacency matrix
    """
    n = A.shape[0]
    A = A + sp.identity(n)
    degs = A.dot(np.ones(n))
    inv_degs = np.power(degs, -1)
    D = sp.diags(inv_degs)
    A_normalized = D.dot(A)

    return A_normalized

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """
    Function that converts a Scipy sparse matrix to a sparse Torch tensor
    """
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

# Load graphs
edge_per_graph, adj, features, edge_features = load_data() 

# Normalize adjacency matrices
adj = [normalize_adjacency(A) for A in adj]



KeyboardInterrupt: 

In [ ]:
print(' '.join(list(map(str,features[0][30][23:]))))

0.0 0.0 2.35 9.87 7.0 6.11 89.0 4.0 11.5 0.0 8.1 4.34 78.0 0.62 -0.4 -0.5 1.8 12.97 0.44 0.616 0.61 0.31 0.1 0.3 5.33 1.36 0.39 0.62 1.94 1.15 -0.3 2.1 0.42 0.35 5.1 3.9 7.3 0.38 -0.1 0.5 11.2 6.6 0.38 0.74 0.0 86.599998 0.36 1.42 0.83 0.66 1.489 0.709 0.788 0.824 1.29 0.9 0.77 0.92 0.9 1.0 8.3 8.25 100.0


In [4]:
# # Split data into training and test sets
# adj_train = list()
# features_train = list()
# y_train = list()
# # adj_test = list()
# # features_test = list()
# proteins_test = list()
# with open('data/graph_labels.txt', 'r') as f:
#     for i,line in enumerate(f):
#         t = line.split(',')
#         if len(t[1][:-1]) == 0:
#             proteins_test.append(t[0])
#         #             adj_test.append(adj[i])
#         #             features_test.append(features[i])
#         else:
#         #             adj_train.append(adj[i])
#         #             features_train.append(features[i])
#             y_train.append(int(t[1][:-1]))
    
# print(len(y_train))

In [5]:
# G = nx.Graph(adj[0], nodetype=int)
# print(G.number_of_nodes())
# print(adj[0].shape)
# nnei = [str(x) for x in G.neighbors(1)]
# print(nnei)
# print(' '.join(nnei))
# r = [1]
# print(r, int(r))

In [6]:
fft = features[824]
for i,x in enumerate(fft):
    if len(np.where(x[3:23]==1)[0])==0:
        print(i)

116


In [13]:
def data_text_tranformers_file(path_graph_label):
    with open('../data/format_dataset_model.txt', 'w') as f:
        f.write('4888\n')
    with open(path_graph_label, 'r') as f:
        proteins_test = list()
        adj_test = list()
        features_test = list()
        for i,line in enumerate(tqdm(f, total=6111)):
            t = line.split(',')
            if len(t[1][:-1]) == 0:
                proteins_test.append(t[0])
                adj_test.append(adj[i])
                features_test.append(features[i])
            else:
                adjacency, label = adj[i], int(t[1][:-1])
                G = nx.Graph(adjacency, nodetype=int)
                n_nodes = G.number_of_nodes()
                with open('../data/format_dataset_model.txt', 'a') as f:
                    f.write(f'{n_nodes} {label}\n')
                
                for j in range(n_nodes):
                    neighbors = [str(x) for x in G.neighbors(j)]
                    n_neighbors = len(neighbors)
                    feature_train = features[i][j]
                    if len(np.where(feature_train[3:23]==1))==0:
                        id_node = 0
                    else:
                        id_node = np.where(feature_train[3:23]==1)[0]
                    open('../data/format_dataset_model.txt', 'a').write(f"{id_node[0]} {n_neighbors} {' '.join(neighbors)}\n")
                        #{' '.join(list(map(str, feature_train[23:])))}
                        
path_graph_label = '../data/graph_labels.txt'                     
data_text_tranformers_file(path_graph_label)

  0%|          | 0/6111 [00:00<?, ?it/s]

KeyboardInterrupt: 